In [52]:
import os
import PIL.Image as Image
import dlib
# import matplotlib.pyplot as plt
import datetime

In [53]:
import torch
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim

import tensorboardX

In [6]:
epochs = 10
batch_size = 16

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [25]:
data_root = 'images/dataset'
save_path = 'images/cropped_faces'

In [10]:
season_dict = {'spring':0, 'summer':1, 'fall':2, 'winter':3}

print(season_dict.keys())
print(season_dict.values())

dict_keys(['spring', 'summer', 'fall', 'winter'])
dict_values([0, 1, 2, 3])


In [11]:
num_files = 0

In [27]:
# for season in season_dict.keys():
#     season_path = os.path.join(data_root, season)
#     for file in os.listdir(season_path):
#         full_path = os.path.join(season_path, file)
#         img = cv2.imread(full_path)
#         if img is None:
#             continue
#         h, w = img.shape[:2]
#         try:
#             faces = detector(img)
#             if len(faces) > 1:
#                 print(f'{len(faces)} faces detected in {full_path}')
#                 # os.remove(full_path)
#                 continue
#             if len(faces) == 0:
#                 print(f'No face detected in {full_path}')
#                 # os.remove(full_path)
#             face = faces[0]
#         except:
#             print(f'No face detected in {full_path}')
#             os.remove(full_path)
#             continue
#         crop = img[max(0, face.top()):min(face.bottom(), h), max(0, face.left()):min(face.right(), w)]
#         crop = cv2.resize(crop, (256, 256))
#         cv2.imwrite(os.path.join(save_path, f'{season_dict[season]}_{num_files}.jpg'), crop)
#         num_files += 1
#     print(f'{num_files} images saved for {season}')
#     num_files = 0
# print(f'Total number of images saved: {len(os.listdir(save_path))}')

No face detected in images/dataset\spring\0.jpg
No face detected in images/dataset\spring\0.jpg
No face detected in images/dataset\spring\10.jpg
No face detected in images/dataset\spring\10.jpg
No face detected in images/dataset\spring\100.jpg
No face detected in images/dataset\spring\100.jpg
No face detected in images/dataset\spring\1000.jpg
No face detected in images/dataset\spring\1000.jpg
No face detected in images/dataset\spring\1001.jpg
No face detected in images/dataset\spring\1001.jpg
No face detected in images/dataset\spring\1002.jpg
No face detected in images/dataset\spring\1002.jpg
No face detected in images/dataset\spring\1003.jpg
No face detected in images/dataset\spring\1003.jpg
No face detected in images/dataset\spring\1004.jpg
No face detected in images/dataset\spring\1004.jpg
No face detected in images/dataset\spring\1005.jpg
No face detected in images/dataset\spring\1005.jpg
No face detected in images/dataset\spring\1007.jpg
No face detected in images/dataset\spring\1

In [29]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [54]:
class FaceDataset(Dataset):
    def __init__(self, n_classes=4, root_dir=save_path, transform=transform):
        self.root_dir = root_dir
        self.transform = transform
        self.files = os.listdir(self.root_dir)
        self.labels = [int(file.split('_')[0]) for file in self.files]
        
        self.n_classes = n_classes
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.files[idx])
        img = Image.open(img_name)
        img = self.transform(img)
        label = self.labels[idx] if self.n_classes == 4 else self.labels[idx] % 2
        # warm = 0, cool = 1
        return img, label

In [55]:
dataset = FaceDataset(n_classes=4)
train_data = DataLoader(dataset, batch_size=batch_size, shuffle=True)
test_data = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [35]:
model = torch.hub.load('pytorch/vision:v0.6.0', 'vgg16', pretrained=True)
model = model.to(device)

Downloading: "https://github.com/pytorch/vision/zipball/v0.6.0" to C:\Users\pomat/.cache\torch\hub\v0.6.0.zip
c:\Users\pomat\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\pomat\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [37]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [38]:
writer = tensorboardX.SummaryWriter()

In [48]:
def train(model, train_data, epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for i, (inputs, labels) in enumerate(train_data):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        writer.add_scalar('train_loss', loss.item(), i*epoch)
        writer.add_scalar('train_accuracy', 100*correct/total, i*epoch)
        
        if i % 10 == 99:
            print(f'[{i+1}, {running_loss/100:.3f}]')
            running_loss = 0.0
            correct = 0
            total = 0
    train_loss = running_loss/len(train_data)
    train_accuracy = 100*correct/total
    print(f'Training {epoch+1}/{epochs}: {train_loss:.3f}, Training Accuracy: {train_accuracy:.3f}')
    
    return train_loss, train_accuracy

In [49]:
def test(model, test_data, epoch):
    model.eval()
    
    correct = 0
    total = 0
    total_loss = 0
    
    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(test_data):
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            
            writer.add_scalar('test_accuracy', 100*correct/total, batch_idx*epoch)
            writer.add_scalar('test_loss', total_loss/(batch_idx+1), batch_idx*epoch)
    
    test_loss = total_loss/len(test_data)
    test_accuracy = 100*correct/total
    print(f'Testing {epoch+1}/{epochs}: {test_loss:.3f}, Testing Accuracy: {test_accuracy:.3f}')
    
    return test_loss, test_accuracy

In [50]:
# epoch-wise losses and accuracies
# batch-wise losses and accuracies are logged in tensorboard
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []

In [61]:
for epoch in range(epochs):
    train_loss, train_acc = train(model, train_data, epoch)
    test_loss, test_acc = test(model, test_data, epoch)
    
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    test_losses.append(test_loss)
    test_accuracies.append(test_acc)
    time = datetime.datetime.now().strftime('%H:%M:%S')
    # torch.save(model.state_dict(), f'models/vgg_{epoch}_{time}.pth')
    model_name = os.path.join('models', f'vgg_{epoch}_{time}.pth')

Training 1/10: 1.182, Training Accuracy: 47.351
Testing 1/10: 1.376, Testing Accuracy: 33.791
Training 2/10: 1.516, Training Accuracy: 31.593
Testing 2/10: 1.374, Testing Accuracy: 33.791
Training 3/10: 1.372, Training Accuracy: 32.319
Testing 3/10: 1.364, Testing Accuracy: 33.791
Training 4/10: 1.373, Training Accuracy: 33.281
Testing 4/10: 1.369, Testing Accuracy: 33.791
Training 5/10: 1.374, Training Accuracy: 32.535
Testing 5/10: 1.366, Testing Accuracy: 33.791
Training 6/10: 1.369, Training Accuracy: 33.104
Testing 6/10: 1.366, Testing Accuracy: 33.791
Training 7/10: 1.370, Training Accuracy: 33.536
Testing 7/10: 1.366, Testing Accuracy: 33.791
Training 8/10: 1.370, Training Accuracy: 32.869
Testing 8/10: 1.371, Testing Accuracy: 33.791
Training 9/10: 1.371, Training Accuracy: 33.575
Testing 9/10: 1.366, Testing Accuracy: 33.791
Training 10/10: 1.368, Training Accuracy: 33.281
Testing 10/10: 1.364, Testing Accuracy: 33.791


In [ ]:
# plt.figure(figsize=(10, 5))
# plt.plot(train_accuracies, label='Training Accuracy')
# plt.plot(test_accuracies, label='Testing Accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.title('Training and Testing Accuracy')
# plt.legend()
# plt.show()

# plt.figure(figsize=(10, 5))
# plt.plot(train_losses, label='Training Loss')
# plt.plot(test_losses, label='Testing Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.title('Training and Testing Loss')
# plt.legend()
# plt.show()